In [ ]:
import sys
sys.path.append("..")

import os

In [4]:
import argparse

s = "--a=1"

parser = argparse.ArgumentParser( )
parser.add_argument("-a", "--a", )

_StoreAction(option_strings=['-a', '--a'], dest='a', nargs=None, const=None, default=None, type=None, choices=None, required=False, help=None, metavar=None)

In [42]:
a = {"a": 1}

In [44]:
class a:
    def __init__(self, p):
        self.p = p

In [45]:
x = a( 1)

In [47]:
x.i = 1

In [50]:
setattr(x, "l", 2)
x.l

2

In [55]:
" ".join(None)

TypeError: can only join an iterable

In [56]:
a = {"a": None}

In [57]:
a.get("a", "b")